In [ ]:
! cp /content/drive/MyDrive/Datasets/utd_mhad/frames.rar .

In [ ]:
! unrar x frames.rar

Streaming output truncated to the last 5000 lines.
Extracting  frames/a26_s6_t4_color/frame_056_015.png                      65%  OK 
Creating    frames/a26_s7_t1_color                                    OK
Extracting  frames/a26_s7_t1_color/frame_016_000.png                      65%  OK 
Extracting  frames/a26_s7_t1_color/frame_018_001.png                      65%  OK 
Extracting  frames/a26_s7_t1_color/frame_020_002.png                      65%  OK 
Extracting  frames/a26_s7_t1_color/frame_022_003.png                      65%  OK 
Extracting  frames/a26_s7_t1_color/frame_024_004.png                      65%  OK 
Extracting  frames/a26_s7_t1_color/frame_026_005.png                      65%  OK 
Extracting  frames/a26_s7_t1_color/frame_028_006.png                      65%  OK 
Extracting  frames/a26_s7_t1_color/frame_030_007.png                      65%  OK 
Extracting  frames/a26_s7_t1_color/frame_032_008

In [ ]:
! pip install torchlib
! pip install scikit-video

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 5.0 MB/s 
  Created wheel for torchlib: filename=torchlib-0.1-py3-none-any.whl size=70557 sha256=25251a8074017c7aa7aa816a03d720f586a3e3d3035de0bcd2772368aaf5f1a9
  Stored in directory: /root/.cache/pip/wheels/fe/99/68/73310e9d0201c2a2739adb2c417da772705ad4b05ab1a147e3
Successfully built torchlib
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 15.3 MB/s 


In [ ]:
"""
Description: 
Created on 12-03-2022 21:34:25
@author: Sirshapan
"""

import os
import argparse
import time
import random
import sys

import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms, datasets
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
import skvideo.io
from skvideo.io import ffprobe

# Insert other imports


import warnings
warnings.filterwarnings("ignore")


class UtdMhad(Dataset):

    def __init__(self,root_dir,train_split_path,transforms):

        self.root_dir = root_dir

        self.train_split = pd.read_csv(train_split_path)["folders"]

        self.transforms = transforms

    def __len__(self):

        return len(self.train_split)
    
    def __getitem__(self, index):

        videoname = self.train_split[index]

        action_id, subject_id, trial_id, _ = videoname.split("_")

        frames_path = os.path.join(self.root_dir,videoname)

        frame_files = os.listdir(frames_path)

        frame_tensor = []

        for file in frame_files:

            frame = cv2.imread(os.path.join(frames_path,file),cv2.IMREAD_GRAYSCALE)
            frame_tensor.append(self.transforms(frame))

        clip = torch.stack(frame_tensor)

        clip = clip.squeeze(1)

        # action_vec = np.zeros(27)
        # action_vec[int(action_id[1:])-1]=1
        # action_vec = torch.tensor(action_vec)

        action_vec = F.one_hot(torch.tensor(int(action_id[1:])-1),num_classes=27)

        return clip[0].unsqueeze(0),clip,action_vec




def main():

    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081))
    ])

    utd = UtdMhad("D:\\UCF\\Coursework\\CAP5415\\course project\\vae\\frames","D:\\UCF\\Coursework\\CAP5415\\course project\\vae\\train_list.csv",transform)



    for i,j,k in utd:

        print(f"{i.shape}, {j.shape} => {k.shape}")
        print(j.shape)
        break

if __name__=="__main__":

    main()








    



FileNotFoundError: ignored

In [ ]:
"""
Description: 
Created on 12-03-2022 21:32:40
@author: Sirshapan
"""

import os
import argparse
import time
import random
import sys

import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
from model_v2 import AutoEncoder as autoenc
from tqdm import tqdm
import torchvision


# Insert other imports


import warnings
warnings.filterwarnings("ignore")

def train(model, device, train_loader, optimizer, criterion):

    model.train()

    losses = []

    for batch_idx, batch_sample in tqdm(enumerate(train_loader)):

        input_frame, output_vid, action_id = batch_sample

        input_frame = input_frame.to(device)
        output_vid = output_vid.to(device)
        action_id = action_id.to(device)

        output = model(input_frame,action_id)

        # print(output.shape)
        # print(output_vid.shape)

        loss = criterion(output,output_vid)

        loss.backward()

        optimizer.step()

        losses.append(loss.item())

    train_loss = float(np.mean(losses))

    print(f"Train set: Averate Loss: {train_loss}")

    return train_loss




def test(model, device, test_loader, criterion, writer_fake, writer_real):

    pass

    # model.eval()

    # losses = []

    # with torch.no_grad():

    #     for batch_idx, batch_sample in tqdm(enumerate(test_loader)):

    #         input_frame, output_vid, action_id = batch_sample

    #         input_frame = input_frame.to(device)
    #         output_vid = output_vid.to(device)
    #         action_id = action_id.to(device)

    #         output = model(input_frame,action_id)

    #         loss = criterion(output, output_vid)

    #         losses.append(loss.item())

    #     fake_img_grid = torchvision.utils.make_grid(output,normalize=True)
    #     real_img_grid = torchvision.utils.make_grid(output_vid,normalize=True)



def main(FLAGS):

    use_cuda = torch.cuda.is_available()

    device = torch.device("cuda" if use_cuda else "cpu")
    print(f"Torch device selected: {device}")

    # Insert tensorboard codes

    # Transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((16,16)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081))
    ])

    # Fetch the dataset

    train_dataset = UtdMhad("frames","train_list.csv",transform)
    

    # Get the dataloader

    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    # Fetch the model

    model = autoenc().to(device=device)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(),lr = 0.03)

    step = 1

    for epoch in range(1, FLAGS.num_epochs+1):

        print(f"Epoch: {epoch}/{FLAGS.num_epochs}")

        train_loss = train(model, device, train_dataloader, optimizer, criterion)

        torch.save(model.state_dict(), "checkpoint_"+str(epoch)+".pth")

        # break

        # test_loss = test(model, device, test_loader, criterion)

    print("Training and evaluation finished :-)")



if __name__=="__main__":

    parser = argparse.ArgumentParser("Auto Encoder")

    parser.add_argument("--batch_size",type=int, default=8)
    parser.add_argument("--num_epochs",type=int, default=200)

    FLAGS = None
    FLAGS, unparsed = parser.parse_known_args()

    print(FLAGS)

    main(FLAGS)




Namespace(batch_size=8, num_epochs=200)
Torch device selected: cuda
Epoch: 1/200


23it [00:54,  2.37s/it]


Train set: Averate Loss: 0.5353736074074454
Epoch: 2/200


23it [00:53,  2.32s/it]


Train set: Averate Loss: 0.40147079462590424
Epoch: 3/200


...

Train set: Averate Loss: 0.4015862008799677
Epoch: 20/200


23it [00:53,  2.33s/it]


Train set: Averate Loss: 0.4015070951503256
Epoch: 21/200


12it [00:29,  2.48s/it]


KeyboardInterrupt: ignored